In [0]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pickle
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.optim import lr_scheduler

In [54]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  

cuda:0


In [55]:
from google.colab import drive
drive.mount('/content/gdrive/')
drive_dir = '/content/gdrive/My Drive/COGS181 Datasets/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
# model_name = "resnet"# "alexnet", "densenet", 'vgg', 'resnet'
# num_epochs = 25 # reduce num_epochs to see if solve overfitting
# batch_size = 32
# num_workers = 2
# feature_extract = True
# datafile = 'fma'

In [0]:
class Dataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx][0]
        y = self.data[idx][1]
        
        if self.transform:
            x = self.transform(x)
        
        x = torch.Tensor(x)

        return (x, y)

In [0]:
def data_preparation(datafile):
    if datafile == 'fma':
        # FMA
        FMA_data = pickle.load(open(drive_dir + 'FMA/FMA_165x32_normalized.train', 'rb'))
        FMA_trainset = Dataset(FMA_data)
        FMA_trainsize = len(FMA_trainset)
        FMA_trainloader = torch.utils.data.DataLoader(FMA_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        FMA_data = pickle.load(open(drive_dir + 'FMA/FMA_165x32_normalized.test', 'rb'))
        FMA_testset = Dataset(FMA_data)
        FMA_testsize = len(FMA_testset)
        FMA_testloader = torch.utils.data.DataLoader(FMA_testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        FMA_classes = ['Electronic', 'Hip-Hop', 'Rock', 'Folk', 'Punk', 'Old-Time / Historic', 'Experimental', 'Soundtrack', 'Classical', 'Pop', 'Jazz', 'Avant-Garde', 'Psych-Rock', 'Indie-Rock', 'Ambient Electronic', 'Alternative', 'International', 'Instrumental', 'Trip-Hop', 'Metal']
        return FMA_trainloader, FMA_testloader, FMA_classes, FMA_trainsize, FMA_testsize
    elif datafile == 'gtzan':
        # GTZAN
        GTZAN_data = pickle.load(open(drive_dir + 'GTZAN/GTZAN_165x32_normalized.train', 'rb'))
        GTZAN_trainset = Dataset(GTZAN_data)
        GTZAN_trainsize = len(GTZAN_trainset)
        GTZAN_trainloader = torch.utils.data.DataLoader(GTZAN_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        GTZAN_data = pickle.load(open(drive_dir + 'GTZAN/GTZAN_165x32_normalized.test', 'rb'))
        GTZAN_testset = Dataset(GTZAN_data)
        GTZAN_testsize = len(GTZAN_testset)
        GTZAN_testloader = torch.utils.data.DataLoader(GTZAN_testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        GTZAN_classes = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']
        return GTZAN_trainloader, GTZAN_testloader, GTZAN_classes, GTZAN_trainsize, GTZAN_testsize
    elif datafile =='pmc':
      # PMC
        PMC_data = pickle.load(open(drive_dir + 'PMC/PMC_165x32_normalized.train', 'rb'))
        PMC_trainset = Dataset(PMC_data)
        PMC_trainsize = len(PMC_trainset)
        PMC_trainloader = torch.utils.data.DataLoader(PMC_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
        PMC_data = pickle.load(open(drive_dir + 'PMC/PMC_165x32_normalized.test', 'rb'))
        PMC_testset = Dataset(PMC_data)
        PMC_testsize = len(PMC_testset)
        PMC_testloader = torch.utils.data.DataLoader(PMC_testset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
        PMC_classes = ['sad', 'happy', 'scary', 'peaceful']
        return PMC_trainloader, PMC_testloader, PMC_classes, PMC_trainsize, PMC_testsize

    else: 
        # RAVDESS
        RAVDESS_data = pickle.load(open(drive_dir + 'RAVDESS/RAVDESS_165x32_normalized.train', 'rb'))
        RAVDESS_trainset = Dataset(RAVDESS_data)
        RAVDESS_trainsize = len(RAVDESS_trainset)
        RAVDESS_trainloader = torch.utils.data.DataLoader(RAVDESS_trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        RAVDESS_data = pickle.load(open(drive_dir + 'RAVDESS/RAVDESS_165x32_normalized.test', 'rb'))
        RAVDESS_testset = Dataset(RAVDESS_data)
        RAVDESS_testsize = len(RAVDESS_testset)
        RAVDESS_testloader = torch.utils.data.DataLoader(RAVDESS_testset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        RAVDESS_classes = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']
        return RAVDESS_trainloader, RAVDESS_testloader, RAVDESS_classes, RAVDESS_trainsize, RAVDESS_testsize


In [0]:
model = ["resnet", 'vgg']# "alexnet", "densenet", 'vgg', 'resnet'
ep = [30] # reduce num_epochs to see if solve overfitting
bs = [32]
num_workers = 2
feature = [True, False]
f = ['pmc']

In [0]:
# for model in model_name:
#   for f in datafile:
#     for ep in num_epochs:
#       for bs in batch_size:
#         for k in range(3):  # cross/val
#           trainloader, testloader, classes, trainsize, testsize = data_preparation(datafile)
#           num_classes = len(classes)
#           model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
#           model_ft.to(device)
#           # Observe that all parameters are being optimized

#           optimizer_ft = optim.Adam(params_to_update, lr=0.001)
#           # Setup the loss fxn
#           criterion = nn.CrossEntropyLoss()

#           # Train and evaluate
#           model_ft, avg_loss, avg_acc = train_model(model_ft, trainloader, 
#                                       criterion, optimizer_ft, epochs=num_epochs)
#           print('training accuracy: ', model, f,' ', ep,' ' bs,' ', k)
#           # Get test accuracy.
#           correct_net = 0
#           correct_cnn = 0
#           total = 0
#           with torch.no_grad():
              
#               for data in testloader:
#                   images, labels = data
#                   images, labels = images.to(device), labels.to(device)

#                   outputs_cnn = model_ft(images)

#                   _, predicted_cnn = torch.max(outputs_cnn.data, 1)
#                   total += labels.size(0)

#                   correct_cnn += (predicted_cnn == labels).sum().item()


          # print('pre_trained: test acc: %d %%' % (
          #     100 * correct_cnn / total))


In [0]:

# trainloader, testloader, classes, trainsize, testsize = data_preparation(datafile)
# num_classes = len(classes)

In [0]:
def train_model(model, dataloaders,criterion, opt,epochs):
    since = time.time()
    avg_loss = []
    best_model_wts = copy.deepcopy(model.state_dict())
    avg_acc = []
    print_freq = 100
#     model = model.cuda()
    for epoch in range(epochs):
        # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        # print('-' * 10)
        running_loss = 0.0
        running_acc = 0.0

        
        for i, data in enumerate(dataloaders):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            opt.zero_grad()
             # forward
            # track history if only in train
            
            outputs = model(inputs)
            _, preds = torch.max(outputs,1)
            loss = criterion(outputs, labels)

            loss.backward()
            opt.step()
        # statistics
            running_loss += loss.item()*inputs.size(0)
            running_acc += torch.sum(preds == labels.data)
#       
        
        losses = running_loss / trainsize
        acc = running_acc / trainsize
        # print('[epoch: {}, i: {:5d}] avg mini-batch loss: {:.3f}, acc: {:.3f}'.format(
            # epoch, i, losses, acc))
        avg_loss.append(losses)
        avg_acc.append(acc)
        running_loss = 0.0
        running_acc = 0.0      
            
    time_elapsed = time.time() - since
    # print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
#     print('Best val Acc: {:4f}'.format(best_acc))

    """this is only useful if we want to combine training and eval states"""
    # load best model weights
#     model.load_state_dict(best_model_wts)
    return model, avg_loss, avg_acc

In [0]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg16_bn(pretrained=use_pretrained)
        
        """this is uncomment if we want to freeze the inner layer"""
        # freeze inner layer
        if feature_extract:
          for param in model_ft.parameters():
              param.requires_grad = False  
        
        model_ft.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224 # what does it mean?
    
    
    elif model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
#         set_parameter_requires_grad(model_ft, feature_extract)
        if feature_extract:
          for param in model_ft.parameters():
              param.requires_grad = False  
        model_ft.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
# print(model_ft)

In [0]:
# model_ft = model_ft.to(device)

In [0]:
# Gather the parameters to be optimized/updated in this run. 
# params_to_update = model_ft.parameters()
# print("Params to learn:")
# if feature_extract:
#     params_to_update = []
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             params_to_update.append(param)
#             print("\t",name)
# else:
#     for name,param in model_ft.named_parameters():
#         if param.requires_grad == True:
#             print("\t",name)

# # Observe that all parameters are being optimized

# optimizer_ft = optim.Adam(params_to_update, lr=0.001)
# # Setup the loss fxn
# criterion = nn.CrossEntropyLoss()

# # Train and evaluate
# model_ft, avg_loss = train_model(model_ft, trainloader, 
#                              criterion, optimizer_ft, epochs=num_epochs)

In [0]:
# plt.plot(avg_loss, 'b', label='default')
# print_freq = 100
# plt.xlabel('mini-batch index / {}'.format(print_freq))
# plt.ylabel('avg. mini-batch loss')
# plt.legend()
# plt.show()

In [0]:
# # Get test accuracy.
# correct_net = 0
# correct_cnn = 0
# total = 0
# with torch.no_grad():
    
#     for data in testloader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         outputs_cnn = model_ft(images)

#         _, predicted_cnn = torch.max(outputs_cnn.data, 1)
#         total += labels.size(0)

#         correct_cnn += (predicted_cnn == labels).sum().item()


# print('pre_trained: Accuracy of the network on the 10000 test images: %d %%' % (
#     100 * correct_cnn / total))

In [68]:
for model_name in model:
  for datafile in f:
    for feature_extract in feature:
      for num_epochs in ep:
        for batch_size in bs:
        
          trainloader, testloader, classes, trainsize, testsize = data_preparation(datafile)
          num_classes = len(classes)
          model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)
          model_ft.to(device)
          # Observe that all parameters are being optimized
          params_to_update = model_ft.parameters()
          optimizer_ft = optim.Adam(params_to_update, lr=0.001)
          # Setup the loss fxn
          criterion = nn.CrossEntropyLoss()

          # Train and evaluate
          model_ft, avg_loss, avg_acc = train_model(model_ft, trainloader, 
                                      criterion, optimizer_ft, epochs=num_epochs)
          print('feature extraction: ', feature_extract ,' training accuracy: ', 'model name: ', model_name,' data: ', datafile,' epoch: ', num_epochs,' batch_size: ', batch_size,' fold: ', k, ' train_acc: ', avg_acc[-1])
          # Get test accuracy.
          correct_net = 0
          correct_cnn = 0
          total = 0
          with torch.no_grad():
              
              for data in testloader:
                  images, labels = data
                  images, labels = images.to(device), labels.to(device)

                  outputs_cnn = model_ft(images)

                  _, predicted_cnn = torch.max(outputs_cnn.data, 1)
                  total += labels.size(0)

                  correct_cnn += (predicted_cnn == labels).sum().item()

          print('pre_trained: test acc: %d %%' % (100 * correct_cnn / total))


feature extraction:  True  training accuracy:  model name:  resnet  data:  pmc  epoch:  30  batch_size:  32  fold:  1  train_acc:  tensor(0.7381, device='cuda:0')
pre_trained: test acc: 78 %
feature extraction:  False  training accuracy:  model name:  resnet  data:  pmc  epoch:  30  batch_size:  32  fold:  1  train_acc:  tensor(0.7619, device='cuda:0')
pre_trained: test acc: 93 %
feature extraction:  True  training accuracy:  model name:  vgg  data:  pmc  epoch:  30  batch_size:  32  fold:  1  train_acc:  tensor(0.7302, device='cuda:0')
pre_trained: test acc: 62 %
feature extraction:  False  training accuracy:  model name:  vgg  data:  pmc  epoch:  30  batch_size:  32  fold:  1  train_acc:  tensor(0.3730, device='cuda:0')
pre_trained: test acc: 40 %
